<a href="https://colab.research.google.com/github/IchottMano/KonIQ-10K-IchottMano/blob/master/InceptionResnetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly-gpu-2.0-preview
except Exception:
  pass
  
import tensorflow as tf
keras = tf.keras
from keras_applications.imagenet_utils import preprocess_input
AUTOTUNE = tf.data.experimental.AUTOTUNE

import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
from glob import glob

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/main.py", line 45, in main
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/__init__.py", line 96, in create_command
    module = importlib.import_module(module_path)
  File "/usr/lib/python3.6/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 994, in _gcd_import
  File "<frozen importlib._bootstrap>", line 971, in _find_and_load
  File "<frozen importlib._bootstrap>", line 955, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 665, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/in

KeyboardInterrupt: ignored

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!unzip -q "gdrive/My Drive/KONIQ_10K/koniq10k.zip"

In [4]:
import pandas as pd

data_dir = pathlib.Path("1024x768")

image_count = len(list(data_dir.glob('*')))
image_count

numbers = pd.read_csv("gdrive/My Drive/KONIQ_10K/scores.csv", index_col="image_name")

def trans(df, label):
  label = label.split("/")[1]
  try:
    return df.loc[label]["MOS"]
  except KeyError:
    return np.NAN

df = pd.DataFrame(list(data_dir.glob('*')), columns=["filename"])
df["filename"] = df["filename"].apply(lambda x: str(x))
df["data"] = df["filename"].apply(lambda x: trans(numbers, x))
df = df.dropna()
df

,filename,data
0,1024x768/665094052.jpg,2.145631
1,1024x768/6340619552.jpg,2.184466
2,1024x768/8596983746.jpg,3.691589
3,1024x768/450789020.jpg,2.769231
5,1024x768/10597114264.jpg,3.657143
...,...,...
10368,1024x768/5627315065.jpg,3.783784
10369,1024x768/11501016504.jpg,3.114286
10370,1024x768/5950051723.jpg,3.663462
10371,1024x768/4904294830.jpg,3.584071


In [5]:
import tensorflow_datasets as tfds
from sklearn import model_selection

BATCH_SIZE = 16
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

train = df.sample(frac=0.8)
test = pd.concat([train, df]).drop_duplicates(keep=False)

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_data_gen = image_generator.flow_from_dataframe(train,
                                                     class_mode='raw',
                                                     y_col='data',
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH))

test_data_gen = image_generator.flow_from_dataframe(test,
                                                     class_mode='raw',
                                                     y_col='data',
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH))


Found 8058 validated image filenames.
Found 2015 validated image filenames.


In [6]:
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)

model = keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False, 
                                                                 weights='imagenet', 
                                                                 input_tensor=None, 
                                                                 input_shape=IMG_SHAPE, 
                                                                 pooling='avg')

219062272/219055592 [==============================] - 7s 0us/step


In [0]:
model.trainable = False

dense_2048 = tf.keras.layers.Dense(2048, activation='relu', name='predictions_2048')
dense_1024 = tf.keras.layers.Dense(1024, activation='relu', name='predictions_1024')
dense_256  = tf.keras.layers.Dense(256, activation='relu', name='predictions_256')
prediction = tf.keras.layers.Dense(1, activation='relu', name='prediction')

new_model = tf.keras.Sequential([
  model,
  dense_2048,
  dense_1024,
  dense_256,
  prediction
])

In [9]:
base_learning_rate = 0.0001
new_model.compile(optimizer="adam",
                  loss=keras.losses.Huber(),
                  metrics=['mean_squared_error'])

new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 1536)              54336736  
_________________________________________________________________
predictions_2048 (Dense)     (None, 2048)              3147776   
_________________________________________________________________
predictions_1024 (Dense)     (None, 1024)              2098176   
_________________________________________________________________
predictions_256 (Dense)      (None, 256)               262400    
_________________________________________________________________
prediction (Dense)           (None, 1)                 257       
Total params: 59,845,345
Trainable params: 5,508,609
Non-trainable params: 54,336,736
_________________________________________________________________


In [10]:
# files.upload("model.h5")
# new_model.load_weights("model.h5")

new_model.fit_generator(
          train_data_gen,
          steps_per_epoch=200,
          epochs=50,
          validation_data=test_data_gen,
          validation_steps=800)

Epoch 1/50
139/200 [===================>..........] - ETA: 57s - loss: 0.3285 - accuracy: 0.0000e+00

KeyboardInterrupt: ignored

In [0]:
new_model.save_weights("model.h5")
files.download("model.h5")